# Ranking system for medical equipment exchange

##### Semi-automated prioritization system to aid in the procurement and replacement of medical equipment using a hierarchical analytical process.
##### HC-UFU | Faculdade de Engenharia Elétrica - UFU 
###### João Vitor Silva, Lívia Gabrielle Vaz Ferreira, Priscila Alves Nunes 

##### 1. Importing Lib

In [23]:
import pandas as pd
import difflib
import pickle

from datetime import datetime
from dateutil.relativedelta import relativedelta
from unidecode import unidecode

##### 2. Reading raw data from equipment belonging to the EAS

In [29]:
import numpy as np

path = 'C://Users//joaov//OneDrive - Universidade Federal de Uberlândia//Graduacao//Eng_Biomed_9_Periodo//Eclin ll//ProjetoFinal//'
file = 'Raw_Data_Filtered_Eqp.csv'

df_raw_data = pd.read_csv(path + file)
df_raw_data.columns = ['Setor', 'Data De Aquisicao', 'Tipo Equipamento', 'Valor', 'Garantia']

# Simulation data TMEF
values_TMEF = ['Baixo', 'Médio', 'Alto']
df_raw_data['Tempo Médio entre Falhas'] = np.random.choice(values_TMEF, size=len(df_raw_data))


df_raw_data.head()

,Setor,Data De Aquisicao,Tipo Equipamento,Valor,Garantia,Tempo Médio entre Falhas
0,UTI ADULTO,11/11/2011,UMIDIFICADOR AQUECEDOR,0,NÃO,Alto
1,UTI NEO,11/11/2011,UMIDIFICADOR AQUECEDOR,0,NÃO,Baixo
2,UTI ADULTO,11/11/2011,MOD CO2 EXALADO,0,NÃO,Baixo
3,UTI ADULTO,11/11/2011,UMIDIFICADOR AQUECEDOR,0,NÃO,Médio
4,UTI ADULTO,11/11/2011,MOD CO2 EXALADO,0,NÃO,Médio


##### 3. Data Processing

- Calculating the age of the equipment as the difference between the current date and the acquisition date in years.
  
- Grouping the type of equipment in each sector, the amount of equipment, and the age of the oldest equipment in the sector.
  
- Filtering of the 'Setor' data standardizing the 260+ identified sectors into 29 pre-defined sectors

In [25]:
##### Agrupando os equipamentos por setor e definindo quantidade de equipamentos e anos no setor

df_raw_data['Data De Aquisicao'] = pd.to_datetime(df_raw_data['Data De Aquisicao'], dayfirst=True)
df_raw_data['Idade do Equipamento'] = df_raw_data['Data De Aquisicao'].apply(lambda x: relativedelta(datetime.now(), x).years)

df_grouped_data = df_raw_data.groupby(['Setor', 'Tipo Equipamento']).agg({'Tipo Equipamento': 'count', 'Idade do Equipamento': 'min', 'Garantia': 'first', 'Tempo Médio entre Falhas':'first'}).rename(columns={'Tipo Equipamento': 'Quantidade de Equipamentos'})
df_grouped_data = df_grouped_data.reset_index().loc[:, ['Setor', 'Tipo Equipamento', 'Quantidade de Equipamentos', 'Idade do Equipamento', 'Garantia', 'Tempo Médio entre Falhas']]


df_grouped_data.head()

,Setor,Tipo Equipamento,Quantidade de Equipamentos,Idade do Equipamento,Garantia,Tempo Médio entre Falhas
0,8889,TERMOMETRO ELETRONICO,1,11,NÃO,Baixo
1,AGENCIA TRANSFUCIONAL,TERMOMETRO ELETRONICO,1,11,NÃO,Baixo
2,AGENCIA TRANSFUSIONAL,AGITADOR LAB,2,9,NÃO,Médio
3,AGENCIA TRANSFUSIONAL,BALANCA ELETRONICA,1,5,NÃO,Médio
4,AGENCIA TRANSFUSIONAL,BALANCA ELETRONICA PRECISAO,1,16,NÃO,Alto


In [26]:
# 262 df_sectors sem filtragem
# 230 setores removendo acentuação 
# 29 setores aplicando filtro

with open('Sectorfilter.pickle', 'rb') as File:
    mapping = pickle.load(File)

df_grouped_data['Setor'] = df_grouped_data['Setor'].apply(lambda texto: unidecode(texto))
df_grouped_data['Setor'] = df_grouped_data['Setor'].replace(mapping, regex=True)

df_processed_data = df_grouped_data.groupby(['Setor', 'Tipo Equipamento']).agg({'Quantidade de Equipamentos': 'sum', 'Idade do Equipamento': 'max', 'Garantia': 'first',  'Tempo Médio entre Falhas':'first'}).reset_index()


df_sectors = df_grouped_data['Setor'].unique().tolist()
df_sectors = pd.DataFrame({'Setor': df_sectors})


File_processed_data = 'Processed_Data_Eqp.csv'
df_processed_data.to_csv(path+File_processed_data, index=False)

df_processed_data.head()

,Setor,Tipo Equipamento,Quantidade de Equipamentos,Idade do Equipamento,Garantia,Tempo Médio entre Falhas
0,AGENCIA TRANSFUSIONAL,AGITADOR LAB,2,9,NÃO,Médio
1,AGENCIA TRANSFUSIONAL,BALANCA ELETRONICA,1,5,NÃO,Médio
2,AGENCIA TRANSFUSIONAL,BALANCA ELETRONICA PRECISAO,1,16,NÃO,Alto
3,AGENCIA TRANSFUSIONAL,BANHO MARIA DESCONGELAMENTO,1,8,NÃO,Alto
4,AGENCIA TRANSFUSIONAL,BANHO MARIA LABORATORIAL,2,0,SIM,Alto


##### 4. Deployment of prioritization logic

In [27]:

df_request = pd.read_csv(path + 'Requests_Eqp.csv')
df_allDataFiltered = pd.read_csv(path + 'Processed_Data_Eqp.csv')
with open('Parameters.pickle', 'rb') as file:
    mapping = pickle.load(file)


# Substituir os nomes dos setores usando a função map()
df_request['Setor'] = df_request['Setor'].map(mapping['Setor'])

# Verificar se o equipamento está presente no DataFrame de equipamentos totais e calcular o parâmetro de idade
df_request['Idade do Equipamento'] = df_request.apply(lambda row: mapping['Idade do Equipamento'][row['Setor'] in df_allDataFiltered[df_allDataFiltered['Idade do Equipamento'] > 10]['Setor'].values], axis=1)

# Calcular os demais parâmetros
df_request['Garantia'] = df_request['Garantia'].map(mapping['Garantia'])
df_request['Classe de Risco'] = df_request['Classe de Risco'].map(mapping['Classe de Risco'])
df_request['Tempo Médio entre Falhas'] = df_request['Tempo Médio entre Falhas'].map(mapping['Tempo Médio entre Falhas'])
df_request['Prioridade interna do setor'] = df_request['Prioridade interna do setor'].map(mapping['Prioridade interna do setor'])
df_request['Impacto do equipamento'] = df_request['Impacto do equipamento'].map(mapping['Impacto do equipamento'])

# Calcular o ranqueamento de prioridades
df_request['Prioridade'] = (df_request['Idade do Equipamento'] + 
                            df_request['Garantia'] + 
                            df_request['Classe de Risco'] +
                            df_request['Tempo Médio entre Falhas'] + 
                            df_request['Prioridade interna do setor'] + 
                            df_request['Impacto do equipamento']) / 6


# Ordenar por prioridade em ordem decrescente
df_request = df_request.sort_values(by='Prioridade', ascending=False)

df_request

,Setor,Nome do Equipamento,Classe de Risco,Valor,Garantia,Tempo Médio entre Falhas,Prioridade interna do setor,Impacto do equipamento,Idade do Equipamento,Prioridade
1,3,RAIO-X,5,100,4,3,3,5,1,3.500000
4,5,SERRA CIRURGICA OSSEA PNEUMATICA OSCILANTE,5,100,4,1,5,5,1,3.500000
3,3,ULTRASSOM,1,100,4,5,3,5,1,3.166667
7,2,BOMBA INFUSAO SERINGA,5,100,4,5,3,1,1,3.166667
6,3,AUTOCLAVE GRANDE PORTE,1,100,4,1,5,5,1,2.833333
10,3,MAMOGRAFO,1,100,4,1,5,5,1,2.833333
8,3,ELETROCARDIOGRAFO,3,100,1,3,5,1,1,2.333333
2,3,GLICOSIMETRO,3,100,4,3,1,1,1,2.166667
5,4,MONITOR MULTIPARAMETRICO,1,100,4,3,3,1,1,2.166667
9,5,MONITOR MULTIPARAMETRICO,1,100,4,1,1,5,1,2.166667
